In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import re
import string
import nltk
from nltk.corpus import stopwords
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer

In [154]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk, os, re, string, collections
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from  imageio import imread
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import multidict as multidict #pip install multidict
import itertools
#import csvtomd
import re
from collections import Counter
#%matplotlib inline
import spacy
from sklearn.preprocessing import MultiLabelBinarizer, Binarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [155]:
train = pd.read_csv('original_data/train.csv')
test  = pd.read_csv('original_data/test.csv')

In [156]:
def getStopwordsList():
    fileNamesList = ["texts/99webtools.txt", "texts/atire_ncbi.txt", "texts/atire_puurula.txt", "texts/azure.txt", 
                 "texts/bbalet.txt", 
                 "texts/bow_short.txt", "texts/choi_2000naacl.txt", "texts/cook1988_function_words.txt", 
                 "texts/corenlp_acronym.txt", 
                 "texts/corenlp_hardcoded.txt", "texts/corenlp_stopwords.txt", "texts/datasciencedojo.txt", 
                 "texts/deeplearning4j.txt", 
                 "texts/dkpro.txt", "texts/mongodb.txt", "texts/galago_inquery.txt", "texts/gate_keyphrase.txt", 
                 "texts/gensim.txt", 
                 "texts/glasgow_stop_words.txt", "texts/indri.txt", "texts/kevinbouge.txt", "texts/lexisnexis.txt",
                 "texts/lingpipe.txt", 
                 "texts/mallet.txt", "texts/mysql_innodb.txt", "texts/mysql_myisam.txt", "texts/galago_rmstop.txt", 
                 "texts/atire_ncbi.txt", 
                 "texts/galago_rmstop.txt", "texts/nltk.txt", "texts/okapiframework.txt", "texts/okapi_cacm_expanded.txt", 
                 "texts/onix.txt", 
                 "texts/ovid.txt", "texts/postgresql.txt", "texts/pubmed.txt", "texts/quanteda.txt", "texts/r_tm.txt", 
                 "texts/ranksnl_large.txt", 
                 "texts/reuters_wos.txt", "texts/rouge_155.txt", "texts/scikitlearn.txt", "texts/smart.txt", 
                 "texts/snowball_expanded.txt", 
                 "texts/spacy.txt", "texts/spark_mllib.txt", "texts/sphinx_mirasvit.txt", "texts/t101_minimal.txt", 
                 "texts/taporware.txt", 
                 "texts/terrier.txt", "texts/tonybsk_1.txt", "texts/tonybsk_6.txt", "texts/voyant_taporware.txt", 
                 "texts/weka.txt", 
                 "texts/xapian.txt", "texts/xpo6.txt", "texts/zettair.txt"]
    stopwordsList = []
    for fileName in fileNamesList:
        file = open(fileName, "r", encoding="utf8")
        for line in file:
            stripped_line = line. strip()
            line_list = stripped_line
            if line_list not in stopwordsList:
                stopwordsList.append(line_list)
        file.close()
    return stopwordsList

stopwordsList = getStopwordsList()

print(len(stopwordsList))

2253


In [157]:
def clean_text_strict(text):
    text = text.lower()
    text = text.replace('\d+', '')
    invalid_chars = ['#','|','@','!','?','-','_','[',']','%','&',':','.',',',"''",'/','https','(','//t',')','http',
                 ';','\'']
    for char in invalid_chars:
        if char in text:
            text = text.replace(char,' ')
    #removes url and tags
    text = re.sub(r"(?:\@|https?\://)\S+", " ",  text)
    return text

In [158]:
# FUNCIONES AUXILIARES

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def prettify2(word_list):
    wl = []
    for word in word_list:
        word = re.sub(r'[^A-Za-z]', " ",  word)
        word = re.sub(r'\b\w\b',' ', word)
        word = re.sub(r'\s+', ' ', word)
        wl.append(word.strip())
    return wl

def prettify(word):
    word = re.sub(r'[^A-Za-z]', " ",  word)
    word = re.sub(r'\b\w\b',' ', word)
    word = re.sub(r'\s+', ' ', word)
    return word

def lemmatize_text2(word_list):
    text = ' '
    text = text.join(word_list)
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

def getText(word_list):
    text = ' '
    text = text.join(word_list)
    return text

def word_finder(x):
    x = set(x)
    return list(unique_frequent_words.intersection(x))

def cleanStopwords(tweet):
    tweet = tweet.split(' ')
    cleanTweet=""
    for word in tweet:
        if word not in stopwordsList:
            cleanTweet = cleanTweet + " " + word
    return cleanTweet


#tweet = "Our Deeds are the Reason of this #earthquake M"

In [159]:
train.text = train["text"].apply(lambda x: clean_text_strict(x))
test.text = test["text"].apply(lambda x: clean_text_strict(x))

In [160]:
train.text

0       our deeds are the reason of this  earthquake m...
1                  forest fire near la ronge sask  canada
2       all residents asked to  shelter in place  are ...
3       13 000 people receive  wildfires evacuation or...
4       just got sent this photo from ruby  alaska as ...
                              ...                        
7608    two giant cranes holding a bridge collapse int...
7609     aria ahrary  thetawniest the out of control w...
7610    m1 94  01 04 utc  5km s of volcano hawaii     ...
7611    police investigating after an e bike collided ...
7612    the latest  more homes razed by northern calif...
Name: text, Length: 7613, dtype: object

In [161]:
train.text = train.text.apply(prettify)
test.text = test.text.apply(prettify)

In [162]:
train.text

0       our deeds are the reason of this earthquake ma...
1                   forest fire near la ronge sask canada
2       all residents asked to shelter in place are be...
3        people receive wildfires evacuation orders in...
4       just got sent this photo from ruby alaska as s...
                              ...                        
7608    two giant cranes holding bridge collapse into ...
7609     aria ahrary thetawniest the out of control wi...
7610               utc km of volcano hawaii co zdtoyd ebj
7611    police investigating after an bike collided wi...
7612    the latest more homes razed by northern califo...
Name: text, Length: 7613, dtype: object

In [163]:
nlp = spacy.load('en', disable=['parser', 'ner'])
def lemmatize_text_v2(text):
    if text == '' or pd.isnull(text):
        return text
    else:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

train.text = train.text.apply(lemmatize_text_v2)
test.text  = test.text.apply(lemmatize_text_v2)

In [164]:
train.text

0       -PRON- deed be the reason of this earthquake m...
1                   forest fire near la ronge sask canada
2       all resident ask to shelter in place be be not...
3         people receive wildfire evacuation order in ...
4       just get send this photo from ruby alaska as s...
                              ...                        
7608    two giant crane hold bridge collapse into near...
7609      aria ahrary thetawniest the out of control w...
7610               utc km of volcano hawaii co zdtoyd ebj
7611    police investigate after an bike collide with ...
7612    the late more home raze by northern california...
Name: text, Length: 7613, dtype: object

In [165]:
train.text = train.text.apply(cleanStopwords)
test.text  = test.text.apply(cleanStopwords)

In [166]:
test.text

0                                          terrible crash
1                                    earthquake city safe
2          forest fire spot pond goose flee street -PRON-
3                    apocalypse lighting spokane wildfire
4                      typhoon soudelor kill china taiwan
                              ...                        
3258     earthquake safety los angeles safety fastener...
3259     storm ri hurricane -PRON- city hit -PRON- yar...
3260                  green derailment chicago utbxlcbiuy
3261                     meg hazardous outlook hwo rbqjhn
3262       cityofcalgary activate -PRON- municipal eme...
Name: text, Length: 3263, dtype: object

In [168]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer=CountVectorizer(analyzer='word',binary=True)
count_vectorizer.fit(train.text)

train_vec = count_vectorizer.fit_transform(train.text)
test_vec = count_vectorizer.transform(test.text)

print(train_vec[7].todense())
print(test_vec[7].todense())

[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]


In [184]:
train_vec

<7613x19203 sparse matrix of type '<class 'numpy.int64'>'
	with 59234 stored elements in Compressed Sparse Row format>

In [169]:
test.text

0                                          terrible crash
1                                    earthquake city safe
2          forest fire spot pond goose flee street -PRON-
3                    apocalypse lighting spokane wildfire
4                      typhoon soudelor kill china taiwan
                              ...                        
3258     earthquake safety los angeles safety fastener...
3259     storm ri hurricane -PRON- city hit -PRON- yar...
3260                  green derailment chicago utbxlcbiuy
3261                     meg hazardous outlook hwo rbqjhn
3262       cityofcalgary activate -PRON- municipal eme...
Name: text, Length: 3263, dtype: object

In [170]:
y=train.target

In [173]:
model.fit(train_vec,y)

MultinomialNB(alpha=1)

In [177]:
X_train, X_test, y_train, y_test=train_test_split(train_vec,y,test_size=0.2,random_state=2020)

In [178]:
predictions = model.predict(X_test)
accuracy = round(accuracy_score(y_test,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 91.00%.


In [179]:
with open("prediction_history.csv", "a") as myfile:
    predicted = model.predict(X_train)
    train_prediction = str(metrics.accuracy_score(y_train,predicted))
    print('Score para x_train: '+ train_prediction)
    predicted = model.predict(X_test)
    test_prediction = str(metrics.accuracy_score(y_test,predicted))
    print('Score para x_test: '+ test_prediction)
    params = str(model)
    print('Hiperparametros: '+ str(model))
    myfile.write(params+','+test_prediction+","+train_prediction+","+str(datetime.datetime.now())+"\n")

Score para x_train: 0.9254515599343186
Score para x_test: 0.9146421536441235
Hiperparametros: MultinomialNB(alpha=1)


In [180]:
predicted = model.predict(test_vec)

In [181]:
predicted

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
sample_submission=pd.read_csv('original_data/sample_submission.csv')

In [182]:
sample_submission.target= predicted

In [183]:
sample_submission.to_csv('just_text_MB_'+str(datetime.datetime.now())+'.csv',index=False)